<style>
    h1 {
        margin-bottom: -.5em;
    }
</style>

# FEMA Disaster Declarations Summaries Dashboard Update Notebook
### An ArcGIS Notebook leveraging the ArcGIS API for Python to automatically update a FEMA-derived custom Dashboard dataset
By Misti Wudtke | misti@aldermaps.com | Portfolio: [aldermaps.com](https://aldermaps.com)

This ArcGIS Notebook is the data update component of my [FEMA Disaster Declaration Summaries ArcGIS Dashboard](https://disasterdeclarations.aldermaps.com/). 

The Notebook runs automatically once per day via a scheduled Task in ArcGIS Online. It connects to openFEMA's API and accesses [FEMA's Disaster Declaration Summaries (v2)](https://www.fema.gov/openfema-data-page/disaster-declarations-summaries-v2), comparing the most recent Summaries with the Summaries that have already been processed and included in the Dashboard. Any new (by declarationDate) Summaries are processed by the Notebook and added to the Dashboard's Declarations Summaries feature layer.

FEMA's Disaster Declaration Summaries comprise multiple rows/observations for a given Disaster Declaration String (e.g., EM-3454-WI), one row/observation for every entity (usually, a county or tribal area, but occassionally an entire state). This Notebook's processing primarily consists of dissolving the boundaries of multiple FEMA Declaration strings. In other words, a single FEMA Declaration String is represented by one _or more_ rows (e.g. counties) in FEMA's data, whereas in this Dashbaord and its underlying data, a single FEMA Declaration String is represented by a single row/geometry. Note that a row's geometry may not be contiguous, i.e. it may comprise a multipart feature.

An exception to this data organization paradigm is FEMA Declaration Strings that encompass multiple entities (e.g., a Declaration string with rows applying to both counties and tribal areas). In this case, a separate row/geometry is created for each Declaration String/entity combination. Different entities (state, county, tribal area) may be viewed separately within the Dashboard via the filter in the Dashboard header.

The data is summarized at the Declaration String level both to facilitate analysis of spatiotemporal patterns in Disaster Declarations Summaries, and for performance reasons.

The ArcGIS Online Dashboard and its components (web map, feature layers) are publicly available, along with this Dashboard, for use by anyone for anything.

* [ArcGIS Dashboard](https://alder.maps.arcgis.com/apps/dashboards/c38d501975e94300a4b53724ff0f8cc8)
* [Dashboard Web Map](https://alder.maps.arcgis.com/apps/mapviewer/index.html?webmap=8b9bb0580a49458a8d1592c9d78f9b85)
* [Dashboard Feature Layer (includes Summaries & Geometries layers)](https://alder.maps.arcgis.com/home/item.html?id=d37c3c2a6f1c4586baad82828bfc3c59)
* [Notebook (ArcGIS Online)](https://alder.maps.arcgis.com/home/item.html?id=31caad975e8348a2bc9406825b958436)
* [Notebook (GitHub)](https://github.com/AlderMaps/arcgis-api-python/blob/main/Disaster_Declarations_Dashboard.ipynb)

### Imports

In [1]:
import requests

from arcgis.gis import GIS
from arcgis.features import manage_data # For dissolving declaration boundaries

import numpy as np
import pandas as pd

import warnings
from urllib3.exceptions import InsecureRequestWarning
warnings.simplefilter("ignore", InsecureRequestWarning)

### Connect to ArcGIS

In [ ]:
gis = GIS("home")

print(f"Logged in as {gis.properties.user.username} successfully.")

Loged in as AlderMaps successfully.


## Part I: Check for new FEMA Disaster Declarations Summaries records

The first section of the notebook loads the necessary datasets and compares them to see whether new Disaster Declaration Summaries have been added by FEMA since the Notebook was last run.

<hr>

### Get Dashboard feature layers (Geometries and Summaries)

The Notebook uses two feature layers (published in a [single feature layer collection](https://alder.maps.arcgis.com/home/item.html?id=d37c3c2a6f1c4586baad82828bfc3c59)):

* The input or "geometries" layer, which the script references to retrieve feature geometries associated with Declaration Summary rows.
* The output or "dashboard" layer, to which the dissolved summary geometries are written. This is the layer that actually appears in the Dashboard.

Both layers are retrieved and converted to spatial data frames (sdf) to facilitate processing with the summaries data from fema, also a pandas dataframe.

In [3]:
dd_id = "d37c3c2a6f1c4586baad82828bfc3c59"
dd_item = gis.content.get(dd_id)
geometries_layer = dd_item.layers[1]
geometries_sdf = geometries_layer.query().sdf

# Production layer; commented out while testing
#dashboard_layer = dd_item.layers[0]

# Test layer; replace with above on conversion to production
test_id = "edb716da51bc4f7882d13d425ad08fd2"
test_item = gis.content.get(test_id)
dashboard_layer = test_item.layers[0]
dashboard_sdf = dashboard_layer.query().sdf

print("Geometry and dashboard feature layers loaded successfully.")

Geometry and dashboard feature layers loaded successfully.


### Hit openFEMA API; convert response to Pandas Dataframe

I have not altered the default number of records returned by the API, which is 1000.

Two filters are applied to the API URL:

* "fyDeclared ge 2013" : Return only values from the field fyDeclared (Fiscal Year Declared) greater than or equal to 2013
    * _(This filter ensures the declarations match the temporal range of the records in my geometries reference layer)_
* "orderby=declarationDate desc" : Order the results (descending) by the declarationDate field ensures the most recent records are returned

In [4]:
api_url = r"https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$filter=fyDeclared ge 2013&$orderby=declarationDate desc"

response = requests.get(api_url)
data = response.json()
summaries_df = pd.DataFrame(data["DisasterDeclarationsSummaries"])

print("Most recent 1000 Disaster Declarations Summaries loaded successfully.")

Most recent 1000 Disaster Declarations Summaries loaded successfully.


### Add and calculate "Entity" field in Summaries dataframe

As described above, this Notebook rolls up FEMA Disaster Declarations Summaries into a single row per FEMA Declaration String + Entity (e.g. county, tribal area) combination. In the Summaries dataset, "Entity" as I have defined it for the purpose of my dataset is defined by multiple fields:

* "Entity" is "County or Equivalent" if the field "fipsCountyCode" _is not_ "000"
* "Entity" is "Tribal Area or Equivalent" if the field "designatedArea" _is not_ "Statewide" AND the field "fipsCountyCode" is "000"
* "Entity" is "Statewide" if the field "designatedArea" is "Statewide"

(See [FEMA's description of the fipsCountyCode field](https://www.fema.gov/openfema-data-page/disaster-declarations-summaries-v2#:~:text=FIPS%20three%2Ddigit,cannot%20be%20entered.) for more information on why the above is true.)

The addition and calculation of the Entity field both enables the ability to check for new rows from FEMA, and provides a single field to filter by Entity in the header of the Dashboard.

In [5]:
entity_conditions = [
    summaries_df["fipsCountyCode"] != "000",
    (summaries_df["fipsCountyCode"] == "000") & (summaries_df["designatedArea"] != "Statewide"),
    summaries_df["designatedArea"] == "Statewide"
]

entity_values = ["County or Equivalent", "Tribal Area or Equivalent", "State or Equivalent"]

summaries_df["Entity"] = np.select(entity_conditions, entity_values)

print("Entity field addition completed successfully.")

Entity field addition completed successfully.


### Check whether there are new Disaster Declaration Summaries to be added (not already in the Dashboard)

The Summaries dataframe is compared with the dataframe of records already in the Dashboard (comparison on the two fields "femaDeclarationString" and "Entity"). This is accomplished with a multi-key anti-join. If all Summaries records are matched to a record in the Dashboard, then there are no new records to process, and the Notebook effectively stops with this cell.

In [ ]:
multi_key = ["femaDeclarationString", "Entity"]

pre_adds = summaries_df.merge(dashboard_sdf[multi_key], on=multi_key, how="left", indicator=True)
adds_df = pre_adds[pre_adds["_merge"] == "left_only"].drop(columns=["_merge"])

if adds_df.empty:
    proceed = False
    print("Found no new Declarations to add! We're done here.")
else:
    proceed = True
    print("Found rows to add:\n")
    print(f"adds designatedAreas: {adds_df[['designatedArea', 'state']]}")
    print("\nProceeding to the rest of the Notebook.")

Found rows to add:

adds designatedAreas:          designatedArea state
24      Vernon (County)    MO
25     Madison (County)    IN
26  Washington (County)    IN
27       Kiowa (County)    KS
28      Greene (County)    IN

Proceeding to the rest of the Notebook.


## Part II: Process new FEMA Declaration records and append to existing Dashboard dataset

The code in the second half of the Notebook only execute if new records were found in the call to the FEMA API.

<hr>

### Add and calculate additional fields in the Summaries dataframe for use in merge

"fipsFullCode" and "fipsTribalCode fields: The addition of a full five-digit FIPS code enables a single-column comparison of rows between Summaries and the Dashboard dataframes

"COVID19": This field enables the COVID-19 filter in the Dashboard header, allowing the user to view only COVID, only non-COVID, or all Summaries.

In [7]:
if proceed:

    # Add & calculate FIPS & Tribal FIPS fields
    adds_df["fipsFullCode"] = adds_df["fipsStateCode"] + adds_df["fipsCountyCode"]
    adds_df["fipsTribalCode"] = adds_df["fipsStateCode"] + adds_df["placeCode"]

    # Add & calculate COVID-19 field
    adds_df["COVID19"] = np.where(adds_df["declarationTitle"].str.contains("COVID-19"), "Show only COVID-19", "Show only non-COVID-19")

    print("Field additions completed successfully.")

Field additions completed successfully.


### Convert all pseudo-date fields in Summaries dataframe from string/object to true datetime

All other potentially troublesome fields (e.g. FIPS) were checked in preliminary analysis and determined to be the correct data types (object/string).

In [8]:
if proceed:

    date_columns = ["declarationDate", "incidentBeginDate", "incidentEndDate", "disasterCloseoutDate", "lastIAFilingDate", "lastRefresh"]

    for dc in date_columns:
        adds_df[dc] = pd.to_datetime(adds_df[dc], errors="coerce")

    print("Field conversion to datetime completed successfully.")

Field conversion to datetime completed successfully.


### Set up variables and field mappings dictionary for use in get_geometries function

The Fema Disaster Declaration Summaries retrieved fromm openFEMA's API do not include geometry and cannot be displayed directly on a map. To portray the data on a map, county (or tribal area, state) geometry data must be associated with each record. The spatially enabled dataframe geometries_sdf, created above, contains these geometries.

Each row in the Summaries dataframe will be matched with its corresponding geometry in the geometries dataframe. For counties, the full five-digit FIPS code is used; for states, the two-digit State FIPS is used, and for tribal areas a tribal code is used. The names of the key fields in both datasets are set up below.

In [9]:
if proceed:

    state = "State or Equivalent"
    county = "County or Equivalent"
    tribal = "Tribal Area or Equivalent" 

    state_field = "State_FIPS"
    county_field = "Full_FIPS"
    tribal_field1 = "AIANNHFP1"
    tribal_field2 = "AIANNHFP2"
    tribal_field3 = "AIANNHFP3"

    key_fields_dict = {
        "State_FIPS": "fipsStateCode",
        "Full_FIPS": "fipsFullCode",
        "AIANNHFP1": "fipsTribalCode",
        "AIANNHFP2": "fipsTribalCode",
        "AIANNHFP3": "fipsTribalCode"
    }

### Associate Summaries records with their correct geometries

In this step the FEMA Summaries records retrieved from the API are merged with their corresponding geometries. The merge method is incorporated in a function because its parameters are different fields, depending on the entity (e.g. counties merge on 5-digit FIPS, states on 2-digit FIPS).

First the adds dataframe is filtered to the relevent entity, then the merge is performed. The Tribal geometries data (from US Census Bureau) includes three different fields that correspond to the Summaries "placeCode" field, and the Summaries rows can and do potentially utilize any of these three identifiers. Therefore the Tribal Area Entities must be considered three times.

Finally, all merged dataframes are assembled vertically into a single adds dataframe including geometries.

In [10]:
if proceed:

    df_list = []

    def get_geometries(entity, field, df_list):

        entity_adds_df = adds_df[adds_df["Entity"] == entity]

        if not entity_adds_df.empty:
            # The suffixes argument must be included, with one element of the tuple parameter being an empty string.
            # Otherwise the dissolve will fail because both "Entity" fields are renamed to the defaults of "Entity_x" and "Entity_y".
            merged = entity_adds_df.merge(geometries_sdf, left_on=[key_fields_dict[field]], right_on=field, suffixes=('', '_2'))
            df_list.append(merged)

    get_geometries(state, state_field, df_list)
    get_geometries(county, county_field, df_list)
    get_geometries(tribal, tribal_field1, df_list)
    get_geometries(tribal, tribal_field2, df_list)
    get_geometries(tribal, tribal_field3, df_list)

    # Ignore index is required here, else the conversion to feature collection below fails
    adds_geometries = pd.concat(df_list, ignore_index=True)

    print("Geometry incorporation completed successfully.")

Geometry incorporation completed successfully.


### Aggregate Summaries rows to FEMA Declaration String / Entity level

I now have new / to add Summaries data but it is still at the county level. This cell aggregates both the rows and the associated geometry to the level of the FEMA Declaration String (and Entity field). This is accomplished with the ArcGIS API for Python dissolve_boundaries method. The fields used in the dissolve are derived from the fields present in the current Dashboard layer.

In [11]:
if proceed:

    adds_feature_collection = adds_geometries.spatial.to_feature_collection()

    remove_fields = ["OBJECTID", "Shape__Area", "Shape__Length", "SHAPE"]
    dissolve_fields = [f["name"] for f in dashboard_layer.properties.fields if not f["name"] in remove_fields]

    adds_dissolved = manage_data.dissolve_boundaries(adds_feature_collection, dissolve_fields=dissolve_fields)

    print("Dissolve boundaries completed successfully.")

{"cost": 0.24}


Dissolve boundaries completed successfully.


### Add new features to the Dashboard layer

Finally, the aggregated rows and dissolved geometries are appended to the Dashboard layer via edit_features.

In [12]:
if proceed:
    
    adds_features = adds_dissolved.query().features
    message = dashboard_layer.edit_features(adds_features)

    print("The following features were added successfully:")
    for m in message["addResults"]:
        print(m)

The following features were added successfully:
{'objectId': 254, 'uniqueId': 254, 'globalId': None, 'success': True}
{'objectId': 255, 'uniqueId': 255, 'globalId': None, 'success': True}
{'objectId': 256, 'uniqueId': 256, 'globalId': None, 'success': True}
{'objectId': 257, 'uniqueId': 257, 'globalId': None, 'success': True}
{'objectId': 258, 'uniqueId': 258, 'globalId': None, 'success': True}
{'objectId': 259, 'uniqueId': 259, 'globalId': None, 'success': True}
{'objectId': 260, 'uniqueId': 260, 'globalId': None, 'success': True}
{'objectId': 261, 'uniqueId': 261, 'globalId': None, 'success': True}
{'objectId': 262, 'uniqueId': 262, 'globalId': None, 'success': True}
{'objectId': 263, 'uniqueId': 263, 'globalId': None, 'success': True}
{'objectId': 264, 'uniqueId': 264, 'globalId': None, 'success': True}
{'objectId': 265, 'uniqueId': 265, 'globalId': None, 'success': True}
{'objectId': 266, 'uniqueId': 266, 'globalId': None, 'success': True}
{'objectId': 267, 'uniqueId': 267, 'global